In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import GridSearchCV
import datetime

In [56]:
df_1_in = pd.read_csv(
    "../dataset/2020-02-road-1-in.csv", parse_dates=["datetime"])
df_1_in

,datetime,road_number,km,direction,all_units,inflow_units,outflow_unit,samecell_units,avg_speed,max_speed,avg_traveltime,max_traveltime
0,2020-02-01 00:00:34,1,25,out,1,0,0,1,43,43,0,0
1,2020-02-01 00:00:34,1,26,out,1,0,0,1,71,71,0,0
2,2020-02-01 00:00:34,1,27,out,1,0,0,1,0,0,0,0
3,2020-02-01 00:00:34,1,28,out,2,1,2,1,41,48,0,0
4,2020-02-01 00:00:34,1,29,out,3,3,4,0,75,108,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
22668874,2020-02-29 23:59:14,1,925,out,0,0,1,0,0,0,0,0
22668875,2020-02-29 23:59:14,1,926,out,1,1,0,0,51,51,60,60
22668876,2020-02-29 23:59:14,1,939,out,1,0,0,1,57,57,0,0
22668877,2020-02-29 23:59:14,1,953,out,0,0,1,0,0,0,0,0


In [ ]:
df_1_out = pd.read_csv(
    "../dataset/2020-02-road-1-out.csv", parse_dates=["datetime"])
df_1_out

In [6]:
df_km127 = pd.read_csv("../dataset/latlon_km127.csv")

In [3]:
import joblib
dtree_model = joblib.load('../Models/DecisionTreeModel.joblib')

In [4]:
dtree_model

DecisionTreeClassifier()

In [7]:
def map_traffic_with_latlon(df):
    df['lat'] = df.apply(lambda row: df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lat'].values[0]
                         if len(df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lat'].values) > 0 else 0, axis=1)
    df['lon'] = df.apply(lambda row: df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lon'].values[0]
                         if len(df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lon'].values) > 0 else 0, axis=1)
    return df


In [8]:
header_list = ['datetime', 'road_number', 'km', 'direction', 'all_units', 'inflow_units',
               'outflow_unit', 'samecell_units', 'avg_speed', 'max_speed', 'avg_traveltime', 'max_traveltime']
df_test = pd.read_csv("../dataset/current_celldata.csv",
                         names=header_list, parse_dates=["datetime"])
df_test = df_test[(df_test['road_number'] == 1) | (
    df_test['road_number'] == 2) | (df_test['road_number'] == 7)]
df_test = map_traffic_with_latlon(df_test)
df_test


,datetime,road_number,km,direction,all_units,inflow_units,outflow_unit,samecell_units,avg_speed,max_speed,avg_traveltime,max_traveltime,lat,lon
0,2022-03-01 17:01:46,1,19,in,0,0,1,0,0,0,0,0,13.879176,100.598832
1,2022-03-01 17:01:46,1,25,in,0,0,1,0,0,0,0,0,13.926152,100.626455
2,2022-03-01 17:01:46,1,25,out,5,4,1,1,24,64,67,90,13.926152,100.626455
3,2022-03-01 17:01:46,1,26,in,4,3,1,1,50,63,69,90,13.935066,100.625095
4,2022-03-01 17:01:46,1,26,out,4,3,5,1,50,63,69,90,13.935066,100.625095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4088,2022-03-01 17:01:46,7,121,in,0,0,1,0,0,0,0,0,12.951311,100.947536
4089,2022-03-01 17:01:46,7,122,in,2,2,0,0,31,32,58,59,12.950342,100.938541
4090,2022-03-01 17:01:46,7,122,out,1,1,0,0,89,89,29,29,12.950342,100.938541
4091,2022-03-01 17:01:46,7,123,in,0,0,1,0,0,0,0,0,12.947607,100.929908


In [9]:
num_cols = ['all_units', 'inflow_units',
            'avg_speed', 'max_speed',
            'max_traveltime']
num_cols

['all_units', 'inflow_units', 'avg_speed', 'max_speed', 'max_traveltime']

In [58]:
x_predict = df_1_in[num_cols]
x_predict

,all_units,inflow_units,avg_speed,max_speed,max_traveltime
0,1,0,43,43,0
1,1,0,71,71,0
2,1,0,0,0,0
3,2,1,41,48,0
4,3,3,75,108,0
...,...,...,...,...,...
22668874,0,0,0,0,0
22668875,1,1,51,51,60
22668876,1,0,57,57,0
22668877,0,0,0,0,0


In [59]:
y_predict = dtree_model.predict(x_predict)
y_predict

array([4, 2, 0, ..., 1, 0, 1], dtype=int64)

In [60]:
df_1_in["cluster"] = y_predict
df_1_in

,datetime,road_number,km,direction,all_units,inflow_units,outflow_unit,samecell_units,avg_speed,max_speed,avg_traveltime,max_traveltime,cluster
0,2020-02-01 00:00:34,1,25,out,1,0,0,1,43,43,0,0,4
1,2020-02-01 00:00:34,1,26,out,1,0,0,1,71,71,0,0,2
2,2020-02-01 00:00:34,1,27,out,1,0,0,1,0,0,0,0,0
3,2020-02-01 00:00:34,1,28,out,2,1,2,1,41,48,0,0,4
4,2020-02-01 00:00:34,1,29,out,3,3,4,0,75,108,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22668874,2020-02-29 23:59:14,1,925,out,0,0,1,0,0,0,0,0,0
22668875,2020-02-29 23:59:14,1,926,out,1,1,0,0,51,51,60,60,1
22668876,2020-02-29 23:59:14,1,939,out,1,0,0,1,57,57,0,0,1
22668877,2020-02-29 23:59:14,1,953,out,0,0,1,0,0,0,0,0,0


In [75]:
df_acc = df_1_in[(df["cluster"] == 0) & (df_1_in["datetime"] < pd.Timestamp(2020, 2, 7,13,40))
    & (df_1_in["datetime"] > pd.Timestamp(2020, 2, 7,12,20))]
df_acc

,datetime,road_number,km,direction,all_units,inflow_units,outflow_unit,samecell_units,avg_speed,max_speed,avg_traveltime,max_traveltime,cluster
5020978,2020-02-07 12:20:27,1,73,out,1,0,1,1,7,7,0,0,0
5021013,2020-02-07 12:20:27,1,108,out,2,0,2,2,7,14,0,0,0
5021015,2020-02-07 12:20:27,1,112,out,0,0,1,0,0,0,0,0,0
5021027,2020-02-07 12:20:27,1,124,out,0,0,2,0,0,0,0,0,0
5021029,2020-02-07 12:20:27,1,126,out,0,0,3,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5068733,2020-02-07 13:39:08,1,983,out,0,0,1,0,0,0,0,0,0
5068734,2020-02-07 13:39:08,1,986,out,0,0,1,0,0,0,0,0,0
5068736,2020-02-07 13:39:08,1,988,out,0,0,1,0,0,0,0,0,0
5068738,2020-02-07 13:39:08,1,990,out,3,1,1,2,18,55,40,40,0


In [76]:
df_acc[(df_acc["km"] == 993)]


,datetime,road_number,km,direction,all_units,inflow_units,outflow_unit,samecell_units,avg_speed,max_speed,avg_traveltime,max_traveltime,cluster
5022107,2020-02-07 12:21:39,1,993,out,0,0,1,0,0,0,0,0,0
5039273,2020-02-07 12:50:10,1,993,out,0,0,1,0,0,0,0,0,0
5041672,2020-02-07 12:54:07,1,993,out,1,1,0,0,17,17,60,60,0
5048248,2020-02-07 13:05:31,1,993,out,1,0,0,1,0,0,0,0,0
5048854,2020-02-07 13:06:42,1,993,out,1,0,0,1,0,0,0,0,0
5065886,2020-02-07 13:34:11,1,993,out,0,0,2,0,0,0,0,0,0
